## Extract

In [20]:
import kaggle
import pandas as pd
import sys
sys.path.append('./scripts')
from etl import *

In [ ]:
# Check the encoding of the files
import chardet
!chardetect /Users/nikolaykutukov/book_recommender_case_study/data/Books.csv
!chardetect /Users/nikolaykutukov/book_recommender_case_study/data/Ratings.csv

In [ ]:
DATASET_NAME = "arashnic/book-recommendation-dataset"

kaggle.api.authenticate()
kaggle.api.dataset_download_files(DATASET_NAME, path='../data', unzip=True)

In [ ]:
ratings_path = '../data/Ratings.csv'
books_path = '../data/Books.csv'
# keep only columns needed for the script
ratings = pd.read_csv(ratings_path, sep=',', encoding='utf-8', dtype={'Book-Rating': 'Int64', 'User-ID': 'Int64', 'ISBN': 'str'})
books = pd.read_csv(books_path, sep=',', encoding='utf-8', usecols=['ISBN', 'Book-Title', 'Book-Author'], dtype={'Book-Title': 'str', 'Book-Author': 'str', 'ISBN': 'str'})

In [ ]:
print(books[books['ISBN']=='3442421322'])

In [ ]:
print(ratings.info())
print(books.info())

In [ ]:
print(ratings.describe())
print(books.describe())

In [ ]:
str(books['Book-Title'][books['ISBN']=='3442421322'])


In [ ]:
print(string_normalize(str(books['Book-Title'][books['ISBN']=='3442421322'])))   

In [ ]:
def clean_books(books):
    # Drop rows with missing ISBN
    books.dropna(subset=['ISBN'], inplace=True)
    books = remove_invalid_isbn(books)
    # Remove duplicates based on ISBN
    books.drop_duplicates(subset=['ISBN'], keep='first', inplace=True)
    # Handle missing values - fill NA with a default value or drop rows
    books.fillna({'Book-Author': 'Unknown'}, inplace=True)
    # Ensure ISBNs are valid
    books['ISBN']= books['ISBN'].apply(clean_isbn)
    books = remove_invalid_isbn(books)
    books['Book-Title-Cleaned'] = books['Book-Title'].apply(string_normalize)
    books['Book-Author-Cleaned'] = books['Book-Author'].apply(string_normalize)
    return books.reset_index(drop=True)

def clean_ratings(ratings):
    # Drop rows where User-ID, ISBN or Book-Rating is missing
    ratings = ratings[ratings['Book-Rating']!=0]
    ratings.dropna(subset=['User-ID', 'ISBN', 'Book-Rating'], inplace=True)
    # Drop rows with invalid ratings
    ratings = ratings[(ratings['Book-Rating'] > 0) & (ratings['Book-Rating'] <= 10)]
    # Drop rows with invalid User-ID
    ratings = ratings[ratings['User-ID'] >= 0]
    # Ensure ISBNs are valid
    ratings['ISBN']= ratings['ISBN'].apply(clean_isbn)
    ratings = remove_invalid_isbn(ratings)
    return ratings.reset_index(drop=True)

books_cleaned = clean_books(books)
ratings_cleaned = clean_ratings(ratings)

print(books.info())
print(ratings.info())

In [ ]:
books_cleaned.to_csv('../data/books_cleaned.csv', encoding='utf-8')
ratings_cleaned.to_csv('../data/ratings_cleaned.csv', encoding='utf-8')